(A) 시술 유형 요약

시술_대분류 : 특정 시술 유형 → IUI/ICSI/IVF/DI/Other

BLASTOCYST_포함 : 특정 시술 유형에 BLASTOCYST 포함(0/1)

AH_포함 : 특정 시술 유형에 AH 포함(0/1)



(B) 배아 단계 진행/결측 패턴

배아_stage_missing_count : 배아 관련 21개 컬럼의 결측 개수

배아_stage_all_missing : 21개 전부 결측(0/1) (중간변수, 학습 입력에서는 제외)

배아_이식_여부 : 1 - 배아_stage_all_missing



(C) 재코딩(범주 단순화)

총시술_bin3 : 총 시술 횟수 → 0회 / 1-2회 / 3회 이상 / Unknown

나이_3구간 : 시술 당시 나이 → 34세 이하 / 35-39세 / 40세 이상 / Unknown



(D) 시술 타이밍

Day5_이식_여부 : 배아 이식 경과일 == 5 (문자/결측 → numeric 안전 변환)



(E) 불임 원인 요약

불임_원인_개수 : 불임 원인 17개 컬럼 numeric 변환 후 합(결측=0)

(F) 결측 신호

전체_missing_count : 행 단위 전체 결측 개수

{PGD/PGS/배아이식/난자채취/배아해동}_isna : 각 컬럼 결측 여부(0/1)

(G) “과정 효율/성과” 비율 feature (0 나눗셈 방지)

이식/생성 = 이식된 배아 수 / 총 생성 배아 수

저장/생성 = 저장된 배아 수 / 총 생성 배아 수

해동/저장 = 해동된 배아 수 / 저장된 배아 수

미세주입_배아/난자 = 미세주입 생성 배아 수 / 미세주입 난자 수

미세주입_이식/이식 = 미세주입 배아 이식

In [14]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

import lightgbm as lgb
from catboost import CatBoostClassifier, Pool


# ============================================================
# 0) Load
# ============================================================
TRAIN_PATH = "../data/train.csv"
TEST_PATH  = "../data/test.csv"
SUB_PATH   = "../data/sample_submission.csv"
OUT_PATH   = "../outputs/05_sub_ensemble_oofsafe.csv"

# fallback (로컬/코랩/채점환경 대비)
if not os.path.exists(TRAIN_PATH) and os.path.exists("/mnt/data/train.csv"):
    TRAIN_PATH = "/mnt/data/train.csv"

train = pd.read_csv(TRAIN_PATH)
test  = pd.read_csv(TEST_PATH)
sub   = pd.read_csv(SUB_PATH)

TARGET_COL = "임신 성공 여부"
ID_COL = "ID"
SUB_ID_COL = sub.columns[0]
SUB_PRED_COL = sub.columns[1]


# ============================================================
# 1) Feature engineering (fit 없는 순수 transform만!)
# ============================================================
EMBRYO_STAGE_COLS = [
    "단일 배아 이식 여부","착상 전 유전 진단 사용 여부","배아 생성 주요 이유",
    "총 생성 배아 수","미세주입된 난자 수","미세주입에서 생성된 배아 수",
    "이식된 배아 수","미세주입 배아 이식 수","저장된 배아 수",
    "미세주입 후 저장된 배아 수","해동된 배아 수","해동 난자 수",
    "수집된 신선 난자 수","저장된 신선 난자 수","혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수","기증자 정자와 혼합된 난자 수",
    "동결 배아 사용 여부","신선 배아 사용 여부","기증 배아 사용 여부","대리모 여부",
]

INFERTILITY_COLS = [
    "남성 주 불임 원인","남성 부 불임 원인","여성 주 불임 원인","여성 부 불임 원인",
    "부부 주 불임 원인","부부 부 불임 원인","불명확 불임 원인",
    "불임 원인 - 난관 질환","불임 원인 - 남성 요인","불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인","불임 원인 - 자궁경부 문제","불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도","불임 원인 - 정자 면역학적 요인","불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태"
]

MISS_FLAG_COLS = ["PGD 시술 여부", "PGS 시술 여부", "배아 이식 경과일", "난자 채취 경과일", "배아 해동 경과일"]

def safe_div(a, b):
    # 0/0, x/0 방지
    return np.where(b == 0, 0.0, a / b)

def add_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # 시술_대분류
    def major_procedure(x):
        if pd.isna(x):
            return "Unknown"
        x = str(x)
        if "IUI" in x:  return "IUI"
        if "ICSI" in x: return "ICSI"
        if "IVF" in x:  return "IVF"
        if "DI" in x:   return "DI"
        return "Other"

    df["시술_대분류"] = df["특정 시술 유형"].apply(major_procedure)

    # 보조기술 토큰
    s = df["특정 시술 유형"].astype("object").fillna("Unknown").astype(str)
    df["BLASTOCYST_포함"] = s.str.contains("BLASTOCYST", na=False).astype(int)
    df["AH_포함"]         = s.str.contains("AH", na=False).astype(int)

    # 배아 단계 결측 패턴
    df["배아_stage_missing_count"] = df[EMBRYO_STAGE_COLS].isna().sum(axis=1)
    df["배아_stage_all_missing"]   = (df["배아_stage_missing_count"] == len(EMBRYO_STAGE_COLS)).astype(int)
    df["배아_이식_여부"]           = 1 - df["배아_stage_all_missing"]

    # 총시술_bin3
    def collapse_trials(x):
        if pd.isna(x):
            return "Unknown"
        if x == "0회":
            return "0회"
        if x in ["1회", "2회"]:
            return "1-2회"
        return "3회 이상"
    df["총시술_bin3"] = df["총 시술 횟수"].apply(collapse_trials)

    # 나이_3구간
    def age_group_simple(age):
        if pd.isna(age) or age == "알 수 없음":
            return "Unknown"
        if age == "만18-34세":
            return "34세 이하"
        if age in ["만35-37세", "만38-39세"]:
            return "35-39세"
        return "40세 이상"
    df["나이_3구간"] = df["시술 당시 나이"].apply(age_group_simple)

    # Day5 이식 여부 (문자/결측 안전)
    d = pd.to_numeric(df["배아 이식 경과일"], errors="coerce")
    df["Day5_이식_여부"] = (d == 5).astype(int)

    # 불임 원인 개수 (안전 변환)
    tmp = df[INFERTILITY_COLS].apply(pd.to_numeric, errors="coerce").fillna(0)
    df["불임_원인_개수"] = tmp.sum(axis=1)

    # 전체 결측 개수 (결측 자체가 신호일 때가 많음)
    df["전체_missing_count"] = df.isna().sum(axis=1)

    # 결측 flag
    for c in MISS_FLAG_COLS:
        if c in df.columns:
            df[f"{c}_isna"] = df[c].isna().astype(int)

    # ===== 비율/성공률 계열 (0 나눗셈 방지) =====
    # 숫자형 후보들 안전 변환
    num_candidates = [
        "총 생성 배아 수", "이식된 배아 수", "저장된 배아 수", "해동된 배아 수",
        "미세주입된 난자 수", "미세주입에서 생성된 배아 수", "미세주입 배아 이식 수",
        "미세주입 후 저장된 배아 수", "해동 난자 수", "수집된 신선 난자 수", "혼합된 난자 수"
    ]
    for c in num_candidates:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    if "총 생성 배아 수" in df.columns and "이식된 배아 수" in df.columns:
        df["이식/생성"] = safe_div(df["이식된 배아 수"].fillna(0), df["총 생성 배아 수"].fillna(0))

    if "총 생성 배아 수" in df.columns and "저장된 배아 수" in df.columns:
        df["저장/생성"] = safe_div(df["저장된 배아 수"].fillna(0), df["총 생성 배아 수"].fillna(0))

    if "저장된 배아 수" in df.columns and "해동된 배아 수" in df.columns:
        df["해동/저장"] = safe_div(df["해동된 배아 수"].fillna(0), df["저장된 배아 수"].fillna(0))

    if "미세주입된 난자 수" in df.columns and "미세주입에서 생성된 배아 수" in df.columns:
        df["미세주입_배아/난자"] = safe_div(df["미세주입에서 생성된 배아 수"].fillna(0), df["미세주입된 난자 수"].fillna(0))

    if "이식된 배아 수" in df.columns and "미세주입 배아 이식 수" in df.columns:
        df["미세주입_이식/이식"] = safe_div(df["미세주입 배아 이식 수"].fillna(0), df["이식된 배아 수"].fillna(0))

    return df


train_fe = add_features(train)
test_fe  = add_features(test)


# ============================================================
# 2) Feature/Column split
# ============================================================
drop_cols = [ID_COL, TARGET_COL, "배아_stage_all_missing"]  # 중간 파생 제외
feature_cols = [c for c in train_fe.columns if c not in drop_cols]

X = train_fe[feature_cols].copy()
y = train_fe[TARGET_COL].copy()
X_test = test_fe[feature_cols].copy()

# categorical 후보: object + 명시적 범주 + 파생 범주
explicit_cat = [
    '시술 시기 코드','시술 당시 나이','시술 유형','특정 시술 유형','배란 자극 여부','배란 유도 유형',
    '배아 생성 주요 이유','총 시술 횟수','클리닉 내 총 시술 횟수','IVF 시술 횟수','DI 시술 횟수',
    '총 임신 횟수','IVF 임신 횟수','DI 임신 횟수','총 출산 횟수','IVF 출산 횟수','DI 출산 횟수',
    '난자 출처','정자 출처','난자 기증자 나이','정자 기증자 나이',
    '시술_대분류','총시술_bin3','나이_3구간',
]
cat_cols = [c for c in explicit_cat if c in X.columns]
# object dtype도 범주형으로 포함
cat_cols = sorted(list(set(cat_cols + X.select_dtypes(include=["object"]).columns.tolist())))

num_cols = [c for c in X.columns if c not in cat_cols]


# ============================================================
# 3) Fold-wise preprocessing (규칙 준수 핵심)
#    - numeric: fold train median으로만 결측치 채움
#    - categorical: 문자열화 + 결측은 "Unknown" (test에서 fit 금지)
# ============================================================
def prep_fold(X_tr, X_va, X_te):
    X_tr = X_tr.copy()
    X_va = X_va.copy()
    X_te = X_te.copy()

    # categorical
    for c in cat_cols:
        X_tr[c] = X_tr[c].astype("object").fillna("Unknown").astype(str)
        X_va[c] = X_va[c].astype("object").fillna("Unknown").astype(str)
        X_te[c] = X_te[c].astype("object").fillna("Unknown").astype(str)

    # numeric: train fold median only
    med = X_tr[num_cols].median(numeric_only=True)
    X_tr[num_cols] = X_tr[num_cols].fillna(med)
    X_va[num_cols] = X_va[num_cols].fillna(med)
    X_te[num_cols] = X_te[num_cols].fillna(med)

    return X_tr, X_va, X_te


# ============================================================
# 4) CV: CatBoost + LightGBM 앙상블 (OOF-safe)
# ============================================================
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

oof_cb = np.zeros(len(X))
oof_lgb = np.zeros(len(X))
pred_cb = np.zeros(len(X_test))
pred_lgb = np.zeros(len(X_test))

for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y), 1):
    X_tr_raw, X_va_raw = X.iloc[tr_idx], X.iloc[va_idx]
    y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

    X_tr, X_va, X_te = prep_fold(X_tr_raw, X_va_raw, X_test)

    # ---------
    # CatBoost
    # ---------
    # class weight (불균형 대응)
    pos = (y_tr == 1).sum()
    neg = (y_tr == 0).sum()
    # [class0_weight, class1_weight]
    cb_class_weights = [1.0, (neg / max(pos, 1))]

    train_pool = Pool(X_tr, y_tr, cat_features=cat_cols)
    valid_pool = Pool(X_va, y_va, cat_features=cat_cols)
    test_pool  = Pool(X_te, cat_features=cat_cols)

    cb = CatBoostClassifier(
        loss_function="Logloss",
        eval_metric="AUC",
        iterations=8000,
        learning_rate=0.03,
        depth=8,
        l2_leaf_reg=6.0,
        random_strength=1.0,
        subsample=0.8,
        rsm=0.8,
        class_weights=cb_class_weights,
        random_seed=42,
        verbose=200,
        allow_writing_files=False
    )
    cb.fit(train_pool, eval_set=valid_pool, use_best_model=True, early_stopping_rounds=300)

    oof_cb[va_idx] = cb.predict_proba(X_va)[:, 1]
    pred_cb += cb.predict_proba(X_te)[:, 1] / skf.n_splits

    auc_cb = roc_auc_score(y_va, oof_cb[va_idx])

    # ---------
    # LightGBM
    # ---------
    # LGB는 categorical을 category dtype으로 주는 게 유리
    X_tr_lgb = X_tr.copy()
    X_va_lgb = X_va.copy()
    X_te_lgb = X_te.copy()
    for c in cat_cols:
        X_tr_lgb[c] = X_tr_lgb[c].astype("category")
        X_va_lgb[c] = X_va_lgb[c].astype("category")
        X_te_lgb[c] = X_te_lgb[c].astype("category")

    lgb_params = {
        "objective": "binary",
        "metric": "auc",
        "learning_rate": 0.03,
        "num_leaves": 128,
        "min_data_in_leaf": 80,
        "feature_fraction": 0.85,
        "bagging_fraction": 0.85,
        "bagging_freq": 1,
        "lambda_l1": 0.0,
        "lambda_l2": 0.0,
        "verbosity": -1,
        "seed": 42,
        "feature_fraction_seed": 42,
        "bagging_seed": 42,
        # 불균형 대응
        "scale_pos_weight": (neg / max(pos, 1)),
    }

    dtr = lgb.Dataset(X_tr_lgb, label=y_tr, categorical_feature=cat_cols, free_raw_data=False)
    dva = lgb.Dataset(X_va_lgb, label=y_va, categorical_feature=cat_cols, free_raw_data=False)

    lgbm = lgb.train(
        lgb_params,
        dtr,
        num_boost_round=10000,
        valid_sets=[dva],
        callbacks=[lgb.early_stopping(300, verbose=False)]
    )

    oof_lgb[va_idx] = lgbm.predict(X_va_lgb)
    pred_lgb += lgbm.predict(X_te_lgb) / skf.n_splits

    auc_lgb = roc_auc_score(y_va, oof_lgb[va_idx])

    # ---------
    # Fold summary
    # ---------
    oof_ens_fold = 0.5 * oof_cb[va_idx] + 0.5 * oof_lgb[va_idx]
    auc_ens = roc_auc_score(y_va, oof_ens_fold)

    print(f"[Fold {fold}] AUC  CB: {auc_cb:.6f} | LGB: {auc_lgb:.6f} | ENS(0.5/0.5): {auc_ens:.6f}")

# OOF 전체
auc_cb_all  = roc_auc_score(y, oof_cb)
auc_lgb_all = roc_auc_score(y, oof_lgb)
auc_ens_all = roc_auc_score(y, 0.5 * oof_cb + 0.5 * oof_lgb)

print("\n===== OOF AUC =====")
print(f"CatBoost : {auc_cb_all:.6f}")
print(f"LightGBM : {auc_lgb_all:.6f}")
print(f"Ensemble : {auc_ens_all:.6f}")

# ============================================================
# 5) Submission (sample_submission 컬럼명 그대로)
# ============================================================
pred_test = 0.5 * pred_cb + 0.5 * pred_lgb

out = sub.copy()
out[SUB_ID_COL] = test_fe[ID_COL].values
out[SUB_PRED_COL] = pred_test

os.makedirs(os.path.dirname(OUT_PATH), exist_ok=True)
out.to_csv(OUT_PATH, index=False)
print("Saved:", OUT_PATH)
print(out.head())


/var/folders/zb/7vcy0_ts23q_3pjlgfdd_cl80000gn/T/ipykernel_19569/3237272284.py:196: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_tr[c] = X_tr[c].astype("object").fillna("Unknown").astype(str)
/var/folders/zb/7vcy0_ts23q_3pjlgfdd_cl80000gn/T/ipykernel_19569/3237272284.py:197: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_va[c] = X_va[c].astype("object").fillna("Unknown").astype(str)
/var/folders/zb/7vcy0_ts23q_3pjlgfdd_cl80000gn/T/ipykernel_19569/3237272284.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecate

0:	test: 0.7214947	best: 0.7214947 (0)	total: 230ms	remaining: 30m 40s
200:	test: 0.7375403	best: 0.7375403 (200)	total: 37.3s	remaining: 24m 8s
400:	test: 0.7383654	best: 0.7383744 (388)	total: 1m 13s	remaining: 23m 9s
600:	test: 0.7385177	best: 0.7385438 (550)	total: 1m 51s	remaining: 22m 51s
800:	test: 0.7385036	best: 0.7385746 (740)	total: 2m 30s	remaining: 22m 32s
1000:	test: 0.7382749	best: 0.7385746 (740)	total: 3m 10s	remaining: 22m 8s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.7385745926
bestIteration = 740

Shrink model to first 741 iterations.
[Fold 1] AUC  CB: 0.738575 | LGB: 0.737311 | ENS(0.5/0.5): 0.738530


/var/folders/zb/7vcy0_ts23q_3pjlgfdd_cl80000gn/T/ipykernel_19569/3237272284.py:196: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_tr[c] = X_tr[c].astype("object").fillna("Unknown").astype(str)
/var/folders/zb/7vcy0_ts23q_3pjlgfdd_cl80000gn/T/ipykernel_19569/3237272284.py:197: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_va[c] = X_va[c].astype("object").fillna("Unknown").astype(str)
/var/folders/zb/7vcy0_ts23q_3pjlgfdd_cl80000gn/T/ipykernel_19569/3237272284.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecate

0:	test: 0.7200408	best: 0.7200408 (0)	total: 229ms	remaining: 30m 30s
200:	test: 0.7411778	best: 0.7411818 (199)	total: 37.3s	remaining: 24m 7s
400:	test: 0.7425280	best: 0.7425308 (399)	total: 1m 21s	remaining: 25m 50s
600:	test: 0.7430542	best: 0.7430720 (584)	total: 2m 6s	remaining: 25m 52s
800:	test: 0.7431721	best: 0.7432055 (757)	total: 2m 52s	remaining: 25m 47s
1000:	test: 0.7431660	best: 0.7432155 (841)	total: 3m 38s	remaining: 25m 28s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.7432155377
bestIteration = 841

Shrink model to first 842 iterations.
[Fold 2] AUC  CB: 0.743216 | LGB: 0.741856 | ENS(0.5/0.5): 0.743170


/var/folders/zb/7vcy0_ts23q_3pjlgfdd_cl80000gn/T/ipykernel_19569/3237272284.py:196: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_tr[c] = X_tr[c].astype("object").fillna("Unknown").astype(str)
/var/folders/zb/7vcy0_ts23q_3pjlgfdd_cl80000gn/T/ipykernel_19569/3237272284.py:197: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_va[c] = X_va[c].astype("object").fillna("Unknown").astype(str)
/var/folders/zb/7vcy0_ts23q_3pjlgfdd_cl80000gn/T/ipykernel_19569/3237272284.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecate

0:	test: 0.7140029	best: 0.7140029 (0)	total: 227ms	remaining: 30m 12s
200:	test: 0.7386202	best: 0.7386226 (199)	total: 37.8s	remaining: 24m 24s
400:	test: 0.7399007	best: 0.7399007 (400)	total: 1m 16s	remaining: 24m 14s
600:	test: 0.7404342	best: 0.7404421 (586)	total: 1m 56s	remaining: 23m 48s
800:	test: 0.7405356	best: 0.7405463 (793)	total: 2m 36s	remaining: 23m 24s
1000:	test: 0.7406598	best: 0.7406636 (912)	total: 3m 18s	remaining: 23m 10s
1200:	test: 0.7407279	best: 0.7407324 (1111)	total: 4m	remaining: 22m 42s
1400:	test: 0.7406886	best: 0.7407445 (1254)	total: 4m 39s	remaining: 21m 58s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.7407444894
bestIteration = 1254

Shrink model to first 1255 iterations.
[Fold 3] AUC  CB: 0.740744 | LGB: 0.739544 | ENS(0.5/0.5): 0.740728


/var/folders/zb/7vcy0_ts23q_3pjlgfdd_cl80000gn/T/ipykernel_19569/3237272284.py:196: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_tr[c] = X_tr[c].astype("object").fillna("Unknown").astype(str)
/var/folders/zb/7vcy0_ts23q_3pjlgfdd_cl80000gn/T/ipykernel_19569/3237272284.py:197: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_va[c] = X_va[c].astype("object").fillna("Unknown").astype(str)
/var/folders/zb/7vcy0_ts23q_3pjlgfdd_cl80000gn/T/ipykernel_19569/3237272284.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecate

0:	test: 0.7210183	best: 0.7210183 (0)	total: 230ms	remaining: 30m 41s
200:	test: 0.7371639	best: 0.7371639 (200)	total: 32.8s	remaining: 21m 11s
400:	test: 0.7379093	best: 0.7379093 (400)	total: 1m 6s	remaining: 21m
600:	test: 0.7382593	best: 0.7382611 (599)	total: 1m 43s	remaining: 21m 11s
800:	test: 0.7382748	best: 0.7383430 (689)	total: 2m 23s	remaining: 21m 28s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.738342985
bestIteration = 689

Shrink model to first 690 iterations.
[Fold 4] AUC  CB: 0.738343 | LGB: 0.737755 | ENS(0.5/0.5): 0.738616


/var/folders/zb/7vcy0_ts23q_3pjlgfdd_cl80000gn/T/ipykernel_19569/3237272284.py:196: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_tr[c] = X_tr[c].astype("object").fillna("Unknown").astype(str)
/var/folders/zb/7vcy0_ts23q_3pjlgfdd_cl80000gn/T/ipykernel_19569/3237272284.py:197: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_va[c] = X_va[c].astype("object").fillna("Unknown").astype(str)
/var/folders/zb/7vcy0_ts23q_3pjlgfdd_cl80000gn/T/ipykernel_19569/3237272284.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecate

0:	test: 0.7248541	best: 0.7248541 (0)	total: 286ms	remaining: 38m 6s
200:	test: 0.7388290	best: 0.7388290 (200)	total: 34.3s	remaining: 22m 9s
400:	test: 0.7400602	best: 0.7400602 (400)	total: 1m 7s	remaining: 21m 23s
600:	test: 0.7404861	best: 0.7404861 (600)	total: 1m 44s	remaining: 21m 32s
800:	test: 0.7405598	best: 0.7405665 (767)	total: 2m 23s	remaining: 21m 27s
1000:	test: 0.7404793	best: 0.7405665 (767)	total: 3m 5s	remaining: 21m 33s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.7405664539
bestIteration = 767

Shrink model to first 768 iterations.
[Fold 5] AUC  CB: 0.740566 | LGB: 0.739877 | ENS(0.5/0.5): 0.740801

===== OOF AUC =====
CatBoost : 0.740285
LightGBM : 0.739259
Ensemble : 0.740363
Saved: ../outputs/05_sub_ensemble_oofsafe.csv
           ID  probability
0  TEST_00000     0.005545
1  TEST_00001     0.008930
2  TEST_00002     0.316706
3  TEST_00003     0.236809
4  TEST_00004     0.736707


In [16]:
# ============================================================
# 🔥 최적 가중치 탐색 + Submission
# ============================================================
print("\n===== 최적 가중치 탐색 =====")
best_auc = -1
best_w = None

for w in np.linspace(0, 1, 101):
    oof_mix = w * oof_cb + (1 - w) * oof_lgb
    auc = roc_auc_score(y, oof_mix)
    if auc > best_auc:
        best_auc = auc
        best_w = w

print(f"Best weight (CB): {best_w:.2f}, Best OOF AUC: {best_auc:.6f}")
print(f"Improvement from 0.5/0.5: +{best_auc - auc_ens_all:.6f}")

# 최적 가중치로 test 예측
pred_test = best_w * pred_cb + (1 - best_w) * pred_lgb

# Submission 저장
out = sub.copy()
out[SUB_ID_COL] = test_fe[ID_COL].values
out[SUB_PRED_COL] = pred_test

# 파일명에 가중치와 점수 포함
output_filename = f"../outputs/05_submission_w{best_w:.2f}_CV{best_auc:.6f}.csv"
out.to_csv(output_filename, index=False)

print(f"\n✅ Submission saved!")
print(f"   File: {output_filename}")
print(f"   Weight: CB {best_w:.2f} + LGB {1-best_w:.2f}")
print(f"   CV AUC: {best_auc:.6f}")
print("\n" + out.head().to_string())


===== 최적 가중치 탐색 =====
Best weight (CB): 0.72, Best OOF AUC: 0.740466
Improvement from 0.5/0.5: +0.000103

✅ Submission saved!
   File: ../outputs/05_submission_w0.72_CV0.740466.csv
   Weight: CB 0.72 + LGB 0.28
   CV AUC: 0.740466

           ID  probability
0  TEST_00000     0.005618
1  TEST_00001     0.010509
2  TEST_00002     0.320414
3  TEST_00003     0.240993
4  TEST_00004     0.731953


OOF-safe ensemble: CatBoost + LightGBM

Best weight: CB 0.72 / LGB 0.28

Best OOF AUC: 0.740466

In [17]:
# after CV loop finished
pred_test_cb  = pred_cb
pred_test_lgb = pred_lgb
pred_test_ens = 0.5 * pred_cb + 0.5 * pred_lgb


In [18]:
def save_sub(pred, path):
    out = sub.copy()
    out[SUB_ID_COL] = test_fe[ID_COL].values
    out[SUB_PRED_COL] = pred
    out.to_csv(path, index=False)
    print("Saved:", path)

save_sub(pred_test_cb,  "../outputs/05_sub_cb_only.csv")
save_sub(pred_test_lgb, "../outputs/05_sub_lgb_only.csv")
save_sub(pred_test_ens, "../outputs/05_sub_ens_05_05.csv")


Saved: ../outputs/05_sub_cb_only.csv
Saved: ../outputs/05_sub_lgb_only.csv
Saved: ../outputs/05_sub_ens_05_05.csv
